# Genotype-Phenotype Converters
This file creates the code that allows us to convert back and forth between a bird's genotype and its phenotype

It also contains **Technical Note** sections throughout, that explain the coding and tech details of the code, and give links to external resources

## Save Genotype
First, we create a function that lets us take a dictionary of genes in the bird, and add or update the genotype for a specific gene

**Technical Note:** [A dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) is a type of data in Python that contains a bunch of pieces of smaller data linked to specific words. Data can be saved under a specific name, and then retrived by using that same name, much like definitions in a paper dictionary.

In [1]:
def saveGenotypeToBird(bird, geneName, genotype):
    bird[geneName] = genotype

### saveGenotypeToBird Tests
We can test this code by adding a new gene to a bird, and seeing that the genotype is updated.

**Technical Note:** This block of code is called a unit test, and is used to verify that the main code actually does what we want it to do.  The comments are formatted in [Gherkin](https://cucumber.io/docs/gherkin/), which is a language that is used to describe code tests in a human-readable way

**Technical Note:** If the tests pass, the code block with the tests will not produce any output, so that when we reuse these functions in another notebook, we don't change the UI.  If the tests fail, then the code block will produce errors.

In [2]:
# GIVEN an empty bird genome
bird = {}
# WHEN saveGenotypeToBird is called with a "Fake Gene" genotype of "WT/WT"
saveGenotypeToBird(bird, "Fake Gene", "WT/WT")
# THEN the bird will contain a "WT/WT" genotype for "Fake Gene"
if not bird == {'Fake Gene': 'WT/WT'}:
    raise Exception('Failed to verify genotype converter - Issue with adding gene to empty bird')

In [3]:
# GIVEN a bird genome with a genotype for "Fake Gene"
bird = {'Fake Gene': 'WT/WT'}
# WHEN saveGenotypeToBird is called with a "Fake Gene" genotype of "f/f"
saveGenotypeToBird(bird, "Fake Gene", "f/f")
# THEN the bird will contain a "f/f" genotype for "Fake Gene"
if not bird == {'Fake Gene': 'f/f'}:
    raise Exception('Failed to verify genotype converter - Issue with updating genotype')

## Save Phenotype
Next, we need to create a function that takes a phenotype and saves it as a genotype.  We want to make sure to reset any genotype data that conflicts with the new phenotype, and handle cases where the phenotype may require more complex genotypes. We will break this down into several smaller functions, and link them all up together to form the final function that the peafowl calculator will actually use

### Modify Simple Genes
The first step in saving the phenotype will be to update the genotype for all the autosomal genes that only have two allotypes - wild type or a single possible trait.  To do this we will check each simple gene one-at-a-time.  If the name of the simple gene matches the given phenotype, we will update the genotype for that gene to be homozygous for the trait. However, if the name of the simple gene does NOT match the phenotype, we will delete any existing genotype for that gene, so it does not conflict with the given phenotype.

(Don't worry about phenotypes that require multiple simple genes to be expressed; we will handle those in another function, and possibly deleting the genotypes that might be used in them will not prevent those genotypes from being re-updated when we handle those)

If the phenotype was found during this update, we will return a **True** value, which will allow us to 

**Technical Note:** In order to check all the simple genes, we are using a [for loop](https://docs.python.org/3/tutorial/controlflow.html#for-statements), a piece of code that runs through all items in a list, and runs the same bits of code for each item in the list

In [4]:
def updateSimpleGenesWithPhenotype(bird, phenotype, simpleGenes):
    foundPhenotype = False
    
    # Check simple genes
    for gene in simpleGenes:
        # If the current gene we are checking matches the selected
        # phenotype, set the genotype in the bird equal to homozygous
        # for the trait
        if phenotype == gene['name']:
            bird[gene['name']] = gene['notation'] + '/' + gene['notation']
            foundPhenotype = True
        # If the current gene we are checking does not match the phenotype
        # we should clear the genotype for that gene
        elif gene['name'] in bird:
            del bird[gene['name']]

    return foundPhenotype

#### updateSimpleGenesWithPhenotype Tests
We can test this code by creating a bird with a genotype, calling the function with a different phenotype than the one that would be displayed based on the bird's genotype, and making sure that the bird's genotype is changed to match the new phenotype

Note that in this test we check that the new genotype exists, but we also check that the old genotype was deleted

In [5]:
# GIVEN a bird genome with a genotype for "Fake Gene"
bird = {'Fake Gene': 'f/f'}
# AND a list of simple genes
colorGenes = [
    { 'notation': 'br',      'name': 'Bronze' },
    { 'notation': 'f',       'name': 'Fake Gene' }
]
# WHEN updateSimpleGenes is called
found = updateSimpleGenesWithPhenotype(bird, "Bronze", colorGenes)
# THEN the bird will contain a "br/br" genotype for "Bronze"
if not bird == {'Bronze': 'br/br'}:
    raise Exception('Failed to verify simple gene converter - new phenotype not added')
# AND the bird will NOT contain a genotype for "Fake Gene"
if 'Fake Gene' in bird:
    raise Exception('Failed to verify simple gene converter - old phenotype not removed')

### Modify Multi-Allotype Genes

The second step in saving the phenotype will be to update the genotype for any genes that have multiple possible allotypes. To do this, we first clear the value for the current gene. Then we check each allotype for a specific gene to see if it matches the listed phenotype. If the name of the allotype matches the phenotype, we will update the genotype for the bird to be homozygous for the trait if it is a normal gene.  This code also accounts for sex-linked genes, so when updating the genotype for a sex-linked gene, if the bird is female it adds a "w" instead of the second copy of a homozygous allele  

If the phenotype was found during this update, we will return a True value, which will allow us to exit early when checking multiple genes at once

In [6]:
def updateGeneWithPhenotype(bird, phenotype, sex, geneName, allotypes, geneIsSexLinked = False):
    foundPhenotype = False
    
    # Reset the gene to the default wild type
    if geneName in bird:
        del bird[geneName]
        
    # Check all allotypes to see if they match the phenotype
    for allotype in allotypes:
        if phenotype == allotype['name']:
            if geneIsSexLinked and sex == 'Female':
                bird[geneName] = allotype['notation'] + '/w'
            else:
                bird[geneName] = allotype['notation'] + '/' + allotype['notation']
            foundPhenotype = True

    return foundPhenotype

#### updateGeneWithPhenotype Tests

In [7]:
# GIVEN a female bird genome with a genotype for "Sex-Linked Color"
bird = {'Sex-Linked Color': 'Z(c)/w'}
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# WHEN updateGeneWithPhenotype is called on a female bird for a sex-linked gene
found = updateGeneWithPhenotype(bird, "American Purple", 'Female', 'Sex-Linked Color', sexLinkedColorAllotypes, True)
# THEN the bird will contain a "Z(pl)/w" genotype for "Sex-Linked Color"
if not bird == {'Sex-Linked Color': 'Z(pl)/w'}:
    raise Exception('Failed to verify allotype gene converter - female sex-linked phenotype not added')

In [8]:
# GIVEN a male bird genome with a genotype for "Sex-Linked Color"
bird = {'Sex-Linked Color': 'Z(c)/Z(c)'}
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# WHEN updateGeneWithPhenotype is called on a male bird for a sex-linked gene
found = updateGeneWithPhenotype(bird, "American Purple", 'Male', 'Sex-Linked Color', sexLinkedColorAllotypes, True)
# THEN the bird will contain a "Z(pl)/Z(pl)" genotype for "Sex-Linked Color"
if not bird == {'Sex-Linked Color': 'Z(pl)/Z(pl)'}:
    raise Exception('Failed to verify allotype gene converter - male sex-linked phenotype not added')

In [9]:
# GIVEN a bird genome with a genotype for "Pied"
bird = {'Pied': 'W/W'}
# AND a list of pied allotypes
piedAllotypes = [
    { 'notation': 'p',       'name': 'Dark Pied' },
    { 'notation': 'W',       'name': 'White' }
]
# WHEN updateGeneWithPhenotype is called
found = updateGeneWithPhenotype(bird, "Dark Pied", 'Female', 'Pied', piedAllotypes)
# THEN the bird will contain a "p/p" genotype for "Pied"
if not bird == {'Pied': 'p/p'}:
    raise Exception('Failed to verify allotype gene converter - phenotype not added')

### Putting It All Together

Now that we have created a handful of functions that can handle converting various phenotypes to the appropriate genotype, we can put all the smaller functions together into one larger function that checks all the different kinds of genes and allotypes to get the complete genotype for a specific phenotype

In [24]:
# This converts a phenotype to a genotype in a bird, assuming the phenotype breeds true
def savePhenotypeToBird(bird, phenotype, sex, simpleGenes, multiAllotypeGenes, 
                        sexLinkedAllotypes, multiGeneTraits, 
                        sexAndAutosomalCombos, hetSexTraits):
    # Update the simple genes
    foundPhenotype = updateSimpleGenesWithPhenotype(bird, phenotype, simpleGenes)
    # Update the sex-linked color genes
    for gene in multiAllotypeGenes:
        if updateGeneWithPhenotype(bird, phenotype, sex, gene['name'], gene['allotypes'], gene['sexLinked']):
            foundPhenotype = True

    # At this point, all genes have been cleared except for a matched phenotype
    # If we have found the matching phenotype already, we can stop here
    if foundPhenotype:
        return

    # Save the phenotype data for genotypes that match multiple genes
    for trait in multiGeneTraits:
        if phenotype == trait['name']:
            # for each gene that makes up the multi-gene trait
            for traitGeneNotation in trait['genes']:
                # find that gene in the gene list and save it to the bird
                for gene in simpleGenes:
                    if gene['notation'] == traitGeneNotation:
                        bird[gene['name']] = gene['notation'] + '/' + gene['notation']

    # Save the phoenotype data for genes that require both autosomal and sex-linked trait
    for trait in sexAndAutosomalCombos:
        if phenotype == trait['name']:
            # Save autosomal phenotype
            for gene in simpleGenes:
                if gene['notation'] == trait['autosomalGene']:
                    bird[gene['name']] = gene['notation'] + '/' + gene['notation']

            # Save sex phenotype
            for allotype in sexLinkedAllotypes:
                if allotype['notation'] == trait['sexGene']:
                    if sex == 'Male':
                        bird['Sex-Linked Color'] = allotype['notation'] + '/' + allotype['notation']
                    else: 
                        bird['Sex-Linked Color'] = allotype['notation'] + '/w'
                        
    # Save phenotype for het sex traits
    for trait in hetSexTraits:
        if phenotype == trait['name'] and sex == 'Male':
            bird['Sex-Linked Color'] = trait['alleles'][0] + '/' + trait['alleles'][1]
            

#### savePhenotypeToBird Tests

In [18]:
# GIVEN a bird genome with a genotype for "Fake Gene"
bird = {'Fake Gene': 'f/f'}
# AND a list of simple genes
colorGenes = [
    { 'notation': 'br',      'name': 'Bronze' },
    { 'notation': 'f',       'name': 'Fake Gene' }
]
# WHEN savePhenotypeToBird is called
found = savePhenotypeToBird(bird, "Bronze", 'Male', colorGenes, [], [], [], [], [])
# THEN the bird will contain a "br/br" genotype for "Bronze"
if not bird == {'Bronze': 'br/br'}:
    raise Exception('Failed to verify phenotype-to-gene converter - new phenotype not added')
# AND the bird will NOT contain a genotype for "Fake Gene"
if 'Fake Gene' in bird:
    raise Exception('Failed to verify phenotype-to-gene converter - old simple phenotype not removed')

In [19]:
# GIVEN a female bird genome with a genotype for "Sex-Linked Color"
bird = {'Sex-Linked Color': 'Z(c)/w'}
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# WHEN savePhenotypeToBird is called on a female bird for a sex-linked gene
found = savePhenotypeToBird(bird, "American Purple", 'Female', [], [
    { 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }
], sexLinkedColorAllotypes, [], [], [])
# THEN the bird will contain a "Z(pl)/w" genotype for "Sex-Linked Color"
if not bird == {'Sex-Linked Color': 'Z(pl)/w'}:
    raise Exception('Failed to verify phenotype-to-gene converter - female sex-linked phenotype not added')

In [21]:
# GIVEN a male bird genome with a genotype for "Sex-Linked Color"
bird = {'Sex-Linked Color': 'Z(c)/Z(c)'}
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# WHEN savePhenotypeToBird is called on a male bird for a sex-linked gene
found = savePhenotypeToBird(bird, "American Purple", 'Male', [], [
    { 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }
], sexLinkedColorAllotypes, [], [], [])
# THEN the bird will contain a "Z(pl)/Z(pl)" genotype for "Sex-Linked Color"
if not bird == {'Sex-Linked Color': 'Z(pl)/Z(pl)'}:
    raise Exception('Failed to verify phenotype-to-gene converter - male sex-linked phenotype not added')

In [23]:
# GIVEN a bird genome with a genotype for "Pied"
bird = {'Pied': 'W/W'}
# AND a list of pied allotypes
piedAllotypes = [
    { 'notation': 'p',       'name': 'Dark Pied' },
    { 'notation': 'W',       'name': 'White' }
]
# WHEN savePhenotypeToBird is called
found = savePhenotypeToBird(bird, "Dark Pied", 'Female', [], [
    { 'name': 'Pied', 'allotypes': piedAllotypes, 'sexLinked': False }
], piedAllotypes, [], [], [])
# THEN the bird will contain a "p/p" genotype for "Pied"
if not bird == {'Pied': 'p/p'}:
    raise Exception('Failed to verify phenotype-to-gene converter - phenotype not added')

## Convert Genotype to Phenotype

Finally, we need to create a function that can convert a bird's genotype into a phenotype.  We want to account for scenarios where the genotype matches multiple different phenotypes, since some phenotypes may overlap.  (For example, the genotype for the platinum phenotype overlaps with the genotype of bronze and opal)

### Phenotypes from Simple Genes
The easiest part to convert is the simple genes.  We can simply check each simple gene, and see if the bird contains the correct genotype for that gene.  If it does, then we can update the phenotype of the bird to be the name of the gene.  We also update the list of previous phenotypes so that we can track overlapping phenotypes. (For example, a platinum bird would technically have three phenotypes - Bronze, Opal, and Platinum)  Tracking the overlapping phenotypes will let us determine if the described genotype has ever been recorded before.

**Technical Note:** We can use a [tuple](https://docs.python.org/3/library/stdtypes.html#tuples) here to return multiple different pieces of information from the function, so we can use them elsewhere.  In this case, we are returning both a string that represents the primary phenotype, and a [list](https://docs.python.org/3/library/stdtypes.html#lists) of all other previous phenotypes that we have matched to this bird

In [25]:
def getSimplePhenotypesFromBird(bird, previousPhenotypes, wildType, simpleGenes):
    # Birds are wild type by default
    finalBirdPhenotype = wildType
    
    for gene in simpleGenes:
        if gene['name'] in bird:
            if bird[gene['name']] == gene['notation'] + '/' + gene['notation']:
                # Add the previous bird phenotype to the list, and change the current phenotype
                if finalBirdPhenotype != wildType:
                    previousPhenotypes.append(finalBirdPhenotype)
                
                finalBirdPhenotype = gene['name']

    return (finalBirdPhenotype, previousPhenotypes)

#### getSimplePhenotypesFromBird Tests

In [34]:
# GIVEN a bird genome with a genotype for "Fake Gene"
bird = {'Fake Gene': 'f/f'}
# AND a list of simple genes
colorGenes = [
    { 'notation': 'br',      'name': 'Bronze' },
    { 'notation': 'f',       'name': 'Fake Gene' }
]
# WHEN getSimplePhenotypesFromBird is called
(phenotype, otherPhenotypes) = getSimplePhenotypesFromBird(bird, [], 'Wild Type', colorGenes)
# THEN the phenotype will be "Fake Gene"
if not phenotype == "Fake Gene":
    raise Exception('Failed to verify simple gene phenotype converter - incorrect phenotype found')
# AND the otherPhenotypes will be empty
if not len(otherPhenotypes) == 0:
    raise Exception('Failed to verify simple gene phenotype converter - other phenotypes incorrect')

### Phenotypes from Multi-Allotype Genes

Next, we want to check each potential allotype for more complex genes, and save any phenotypes that match to the list of found phenotypes

In [47]:
def getPhenotypesForGeneFromBird(bird, previousPhenotypes, sex, wildType, geneName, allotypes, geneIsSexLinked = False):
    # Birds are wild type by default
    finalBirdPhenotype = wildType
    
    for allotype in allotypes:
        if geneName in bird and (
            # If the gene is not sex-linked, or the bird is male, check for homozygous genes
            ((not geneIsSexLinked or sex == 'Male') and bird[geneName] == allotype['notation'] + '/' + allotype['notation']) or
            # If the gene is sex linked and the bird is female, check for the female-specific variant
            ((geneIsSexLinked and sex == 'Female') and bird[geneName] == allotype['notation'] + '/w')):
            # Add the previous bird phenotype to the list, and change the current phenotype
            if finalBirdPhenotype != wildType:
                previousPhenotypes.append(finalBirdPhenotype)
                
            finalBirdPhenotype = allotype['name']

    return (finalBirdPhenotype, previousPhenotypes)

#### getPhenotypesForGeneFromBird Tests

In [48]:
# GIVEN a female bird genome with a genotype for "Sex-Linked Color"
bird = {'Sex-Linked Color': 'Z(c)/w'}
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# WHEN getPhenotypesForGeneFromBird is called on a female bird for a sex-linked gene
(phenotype, otherPhenotypes) = getPhenotypesForGeneFromBird(bird, [], 'Female', "Wild Type", 'Sex-Linked Color', 
                                                            sexLinkedColorAllotypes, True)
# THEN the phenotype will be "Cameo"
if not phenotype == "Cameo":
    raise Exception('Failed to verify allotype gene phenotype converter - incorrect phenotype found')
# AND the otherPhenotypes will be empty
if not len(otherPhenotypes) == 0:
    raise Exception('Failed to verify allotype gene phenotype converter - other phenotypes incorrect')

In [49]:
# GIVEN a male bird genome with a genotype for "Sex-Linked Color"
bird = {'Sex-Linked Color': 'Z(c)/Z(c)'}
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# WHEN getPhenotypesForGeneFromBird is called on a male bird for a sex-linked gene
(phenotype, otherPhenotypes) = getPhenotypesForGeneFromBird(bird, [], 'Male', "Wild Type", 'Sex-Linked Color', 
                                                            sexLinkedColorAllotypes, True)
# THEN the phenotype will be "Cameo"
if not phenotype == "Cameo":
    raise Exception('Failed to verify allotype gene phenotype converter - incorrect phenotype found')
# AND the otherPhenotypes will be empty
if not len(otherPhenotypes) == 0:
    raise Exception('Failed to verify allotype gene phenotype converter - other phenotypes incorrect')

In [51]:
# GIVEN a bird genome with a genotype for "Pied"
bird = {'Pied': 'W/W'}
# AND a list of pied allotypes
piedAllotypes = [
    { 'notation': 'p',       'name': 'Dark Pied' },
    { 'notation': 'W',       'name': 'White' }
]
# WHEN savePhenotypeToBird is called
(phenotype, otherPhenotypes) = getPhenotypesForGeneFromBird(bird, [], 'Male', "Wild Type", 'Pied', 
                                                            piedAllotypes)
# THEN the phenotype will be "White"
if not phenotype == "White":
    raise Exception('Failed to verify allotype gene phenotype converter - incorrect phenotype found')
# AND the otherPhenotypes will be empty
if not len(otherPhenotypes) == 0:
    raise Exception('Failed to verify allotype gene phenotype converter - other phenotypes incorrect')

### Determining the Final Phenotype

Now that we have a bunch of functions that can determine parts of the bird's phenotype, we can put it all together to determine the final phenotype for a bird

In [53]:
def getPhenotypeFromBird(bird, sex, wildType, simpleGenes, multiAllotypeGenes):
    # Birds are wild type by default
    finalBirdPhenotype = wildType
    # Because phenotypes are not mutually exclusive, we want to track 
    # all possible phenotypes
    otherBirdPhenotypes = []
    # We also want to track whether or not this phenotype has ever been 
    # observed before
    isUnknownPhenotype = False

    (finalBirdPhenotype, otherBirdPhenotypes) = getSimplePhenotypesFromBird(bird, otherBirdPhenotypes, wildType, simpleGenes)

    for gene in multiAllotypeGenes:
        (finalBirdPhenotype, otherBirdPhenotypes) = getPhenotypesForGeneFromBird(bird, otherBirdPhenotypes, sex, wildType, 
                                                                                 gene['name'], gene['allotypes'], gene['sexLinked'])

    return (finalBirdPhenotype, otherBirdPhenotypes, isUnknownPhenotype)

#### getPhenotypeFromBird Tests

In [55]:
# GIVEN a bird genome with a genotype for "Fake Gene"
bird = {'Fake Gene': 'f/f'}
# AND a list of simple genes
colorGenes = [
    { 'notation': 'br',      'name': 'Bronze' },
    { 'notation': 'f',       'name': 'Fake Gene' }
]
# WHEN getPhenotypeFromBird is called
(phenotype, otherPhenotypes, isUnknown) = getPhenotypeFromBird(bird, "Male", 'Wild Type', colorGenes, [])
# THEN the phenotype will be "Fake Gene"
if not phenotype == "Fake Gene":
    raise Exception('Failed to verify gene-to-phenotype converter - incorrect phenotype found')
# AND the otherPhenotypes will be empty
if not len(otherPhenotypes) == 0:
    raise Exception('Failed to verify gene-to-phenotype converter - other phenotypes incorrect')

In [56]:
# GIVEN a female bird genome with a genotype for "Sex-Linked Color"
bird = {'Sex-Linked Color': 'Z(c)/w'}
# AND a list of allotypes
sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' }
]
# WHEN getPhenotypeFromBird is called on a female bird for a sex-linked gene
(phenotype, otherPhenotypes, isUnknown) = getPhenotypeFromBird(bird, "Female", 'Wild Type', colorGenes, 
    [{ 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }])
# THEN the phenotype will be "Cameo"
if not phenotype == "Cameo":
    raise Exception('Failed to verify allotype gene phenotype converter - incorrect phenotype found')
# AND the otherPhenotypes will be empty
if not len(otherPhenotypes) == 0:
    raise Exception('Failed to verify allotype gene phenotype converter - other phenotypes incorrect')

In [ ]:
    

def getColorFromBird(bird, sex):
    # Birds are wild type by default
    finalBirdColor = "Wild Type"
    # Because colors are not mutually exclusive, we want to track 
    # all possible colors
    otherBirdColors = []
    
    # Set sex to default if not defined so we don't 
    # break things when we try to check the sex-linked color
    if not 'Sex-Linked Color' in bird:
        if sex == 'Male':
            bird['Sex-Linked Color'] = 'Z(WT)/Z(WT)'
        else: 
            bird['Sex-Linked Color'] = 'Z(WT)/w'
    
    # Bird color special cases/Documents/AccessAdminAssist/access-admin-assist-frontend
    if bird['Sex-Linked Color'] == 'Z(c)/Z(pl:c)':
        finalBirdColor = 'Cameo'
    elif bird['Sex-Linked Color'] == 'Z(pl)/Z(pl:c)':
        finalBirdColor = 'American Purple'

    # Check if bird color matches any simple colors
    for gene in colorGenes:
        if gene['name'] in bird:
            if bird[gene['name']] == gene['notation'] + '/' + gene['notation']:
                # Add the previous bird color to a list, and change the final color
                otherBirdColors.append(finalBirdColor)
                finalBirdColor = gene['name']

    # Check if bird color matches any sex-linked colors
    for allotype in sexLinkedColorAllotypes:
        if 'Sex-Linked Color' in bird and ((sex == 'Male' and bird['Sex-Linked Color'] == allotype['notation'] + '/' + allotype['notation']) or
            (sex == 'Female' and bird['Sex-Linked Color'] == allotype['notation'] + '/w')):
            # Add the previous bird color to a list, and change the final color
            otherBirdColors.append(finalBirdColor)
            finalBirdColor = allotype['name']

    # Check bird color matches sexAndAutosomal Combo
    for color in sexAndAutosomalComboColors:
        matchesAll = True
        
        for gene in colorGenes:
            # If the current gene is part of the color
            if (gene['notation'] == color['autosomalGene'] and 
                # and the current gene is NOT in the bird
                (not gene['name'] in bird or 
                bird[gene['name']] != gene['notation'] + '/' + gene['notation'])):
                # then we know the bird does NOT have the color, and we do not 
                # have to check any other genes
                matchesAll = False
                break

        # If we did not find an autosomal gene, we don't need to check the
        # sex-linked genes
        if not matchesAll:
            continue
        
        for allotype in sexLinkedColorAllotypes:
            # If the current gene is part of the color
            if (allotype['notation'] == color['sexGene'] and 
                # and the current gene is NOT in the bird
                (not 'Sex-Linked Color' in bird or 
                 (sex == 'Male' and bird['Sex-Linked Color'] != allotype['notation'] + '/' + allotype['notation']) or
                 (sex == 'Female' and bird['Sex-Linked Color'] != allotype['notation'] + '/w'))):
                matchesAll = False
                break

        # If matchesAllGenes is still true here, we know the bird must have
            # all the needed genes for the color
        if matchesAll:
            # Add the previous bird color to a list, and change the final color
            otherBirdColors.append(finalBirdColor)
            finalBirdColor = color['name']
                

    # Check bird color matches multi-gene color
    for color in multiGeneColors:
        # Make sure all genes needed to make the color are availible in the bird
        matchesAllGenes = True
        for gene in colorGenes:
            # If the current gene is part of the color
            if (gene['notation'] in color['genes'] and 
                # and the current gene is NOT in the bird
                ( not gene['name'] in bird or 
                bird[gene['name']] != gene['notation'] + '/' + gene['notation'])):
                # then we know the bird does NOT have the color, and we do not 
                # have to check any other genes
                matchesAllGenes = False
                break

        # If matchesAllGenes is still true here, we know the bird must have
        # all the needed genes for the color
        if matchesAllGenes:
            # Add the previous bird color to a list, and change the final color
            otherBirdColors.append(finalBirdColor)
            finalBirdColor = color['name']

    for color in hetSexColors:
        if bird['Sex-Linked Color'] == color['alleles'][0] + '/' + color['alleles'][1]:
            # Add the previous bird color to a list, and change the final color
            otherBirdColors.append(finalBirdColor)
            finalBirdColor = color['name']

    return (finalBirdColor, otherBirdColors)